<a href="https://www.kaggle.com/code/adrienloizeau/baseline?scriptVersionId=114174836" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

---

Below is a sample workflow to help you get started. The score obtained from it is around the baseline. At the end there are a few suggestions where you could go from here.

---

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

import numpy as np

from PIL import Image
import matplotlib.pyplot as plt

import glob
from tqdm import tqdm
import time
import os

In [4]:
class config:
    BASE_PATH = "../input/msc-ai-2022/"
    TRAIN_IMG_PATH = "/kaggle/input/msc-ai-2022-small/train_images"
    TRAIN_MASK_PATH = "/kaggle/input/msc-ai-2022-small/train_masks"
    TEST_IMG_PATH = "/kaggle/input/msc-ai-2022-small/test_images"

# 1. Dataloaders

In [5]:
HEIGHT = 256
WIDTH = 256
BATCH_SIZE = 4

In [6]:
from albumentations.pytorch import ToTensorV2

# Create a custom Dataset class
class tensorDataset(Dataset):
    def __init__(self, image_paths: list, mask_paths: list, train: bool, transform = None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.train = train
        self.transform =  A.Compose(
            [
                A.Resize(width=WIDTH, height=HEIGHT),
                A.RandomCrop(width=WIDTH, height=HEIGHT),
                A.Rotate(limit=40, p=0.9, border_mode=cv2.BORDER_CONSTANT),
 #               A.HorizontalFlip(p=0.5),
#                A.VerticalFlip(p=0.1),
                A.Normalize(
                    mean=[0, 0, 0],
                    std=[1, 1, 1],
                    max_pixel_value=255),
                ToTensorV2(),
            ]
        )
        
        
        
    def __getitem__(self, index):
        
        # Select a specific image's path
        img_path  = self.image_paths[index]
        mask_path = self.mask_paths[index]
        
        # Load the image
        img = Image.open(img_path)
        mask = Image.open(mask_path)
        
        image = np.array(img)
        mask = np.array(mask)

        augmentations = self.transform(image=image, mask = mask)
        augmented_img = augmentations["image"]
        augmented_mask = augmentations["mask"]
        
        return augmented_img, augmented_mask

    def __len__(self):
        return len(self.image_paths)

In [7]:
import cv2 
import albumentations as A

# Create a custom Dataset class
class tensorDataset_OLD(Dataset):
    def __init__(self, image_paths: list, mask_paths: list, train: bool):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.train = train
        
        self.transform_img = transforms.Compose([
            transforms.Resize([HEIGHT,WIDTH]),
            #transforms.RandomCrop([HEIGHT,WIDTH]),

            transforms.ToTensor()
        ])
        self.transform_mask = transforms.Compose([
            transforms.Resize([HEIGHT,WIDTH], interpolation = transforms.InterpolationMode.NEAREST), # Important
            #transforms.RandomCrop([HEIGHT,WIDTH]),
#            transforms.RandomHorizontalFlip(p=0.2),
            transforms.ToTensor()
        ])
        
    def __getitem__(self, index):
        
        # Select a specific image's path
        img_path  = self.image_paths[index]
        mask_path = self.mask_paths[index]
        
        # Load the image
        img = Image.open(img_path)
        mask = Image.open(mask_path)
    
        # Apply transformations
        img = self.transform_img(img)
        mask = self.transform_mask(mask)
        
        # Scale the mask from 0-1 range to 0-255 range
        mask = mask * 255  
        
        # Reshape mask from (1, H, W) to (H, W) -> This is because loss function accepts (B, H, W) not (B, 1, H, W)
        mask = mask.squeeze(0)
        
        return img, mask

    def __len__(self):
        return len(self.image_paths)

In [8]:
# Very simple train/test split
train_ratio = 0.8
train_set_last_idx = int(len(glob.glob(config.TRAIN_IMG_PATH + "/*")) * train_ratio)

train_img_paths  = sorted(glob.glob(config.TRAIN_IMG_PATH + "/*"))[:train_set_last_idx]
train_mask_paths = sorted(glob.glob(config.TRAIN_MASK_PATH + "/*"))[:train_set_last_idx]
val_img_paths    = sorted(glob.glob(config.TRAIN_IMG_PATH + "/*"))[train_set_last_idx:]
val_mask_paths   = sorted(glob.glob(config.TRAIN_MASK_PATH + "/*"))[train_set_last_idx:]

# Create datasets
train_dataset = tensorDataset(train_img_paths, train_mask_paths, train=True)
val_dataset   = tensorDataset(val_img_paths, val_mask_paths, train=False)

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory = True)
val_dataloader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory = True)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [9]:
# Test dataloaders
start = time.time()
for batch in tqdm(train_dataloader):
    img_batch, img_mask = batch

for batch in tqdm(val_dataloader):
    img_batch, img_mask = batch
      
end = time.time()
print(f'Seconds needed to load one train + val epoch: {end - start :.3f}')

100%|██████████| 14/14 [00:00<00:00, 18.40it/s]

Seconds needed to load one train + val epoch: 7.556


The dataloaders are extremely slow! It takes around a minute to load one epoch, which can be a huge overhead if you want to train for e.g. 100 epochs. The reason is that the images are quite large (some around 3000x4000) and loading such images takes a lot of time. Ideally, for this dataset you should have dataloaders that load one epoch in a few seconds at max. To achieve this we can do a few things:

1. We could pre-process the images. Reszing the images beforehand to the desired height and width, will significantly decrease the loading time.
2. We could increase the number of workers. However, remember that the more batches you store at once, the less memory you have for other operations.

## 2. Model definition

In [10]:
class BasicBlock_OLD(nn.Module):
    def __init__(self, in_channels: int, forward_expansion: int,  out_channels: int, expand: bool):
        super(BasicBlock, self).__init__()
        """
        A very simple convlution block. Reduces or expands the size of the image by a factor of 2.
        When using batchnorm, you can set bias=False to preceding convolution.
        """
        self.conv1 = nn.Conv2d(in_channels, forward_expansion, 3, stride=1, padding='same', bias=False)
        self.bn1 = nn.BatchNorm2d(forward_expansion)
        self.conv2 = nn.Conv2d(forward_expansion, out_channels, 3, stride=1, padding='same', bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        if expand:
            self.scaling = nn.ConvTranspose2d(out_channels, out_channels, kernel_size=2, stride=2)
        else:
            self.scaling = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
    def forward(self, x):
        x = self.bn1(self.conv1(x))
        x = F.relu(x)
        x = self.bn2(self.conv2(x))
        x = F.relu(x)
        x = self.scaling(x)
        return x

    
class SampleModel(nn.Module):
    def __init__(self, in_channels=3, out_channels=25):
        super(SampleModel, self).__init__()
        
        # Create downsizing pass
        self.down = nn.ModuleList()
        self.down.append(BasicBlock(3, 256, 256, expand=False))
        self.down.append(BasicBlock(256, 512, 512, expand=False))
        self.down.append(BasicBlock(512, 512, 1024, expand=False))
        
        # Create bottleneck
        self.bottleneck = nn.Conv2d(1024, 1024, 3, stride=1, padding='same')
        
        # Create upsizing pass
        self.up = nn.ModuleList()
        self.up.append(BasicBlock(1024, 512, 512, expand=True))
        self.up.append(BasicBlock(512, 512, 256, expand=True))
        self.up.append(BasicBlock(256, 256, out_channels, expand=True))
        
    def forward(self, x):
        for block in self.down:
            x = block(x)
        x = self.bottleneck(x)
        for block in self.up:
            x = block(x)
        return x

In [11]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels: int, forward_expansion: int,  out_channels: int, expand: bool):
        super(BasicBlock, self).__init__()
        """
        A very simple convlution block. Reduces or expands the size of the image by a factor of 2.
        When using batchnorm, you can set bias=False to preceding convolution.
        """
        self.conv1 = nn.Conv2d(in_channels, forward_expansion, 3, stride=1, padding='same', bias=False)
        self.bn1 = nn.BatchNorm2d(forward_expansion)
        self.conv2 = nn.Conv2d(forward_expansion, out_channels, 3, stride=1, padding='same', bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        if expand:
            self.scaling = nn.ConvTranspose2d(out_channels, out_channels, kernel_size=2, stride=2)
        else:
            self.scaling = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
    def forward(self, x):
        x = self.bn1(self.conv1(x))
        x = F.relu(x)
        x = self.bn2(self.conv2(x))
        x = F.relu(x)
        x = self.scaling(x)

        return x

    
class SampleModel(nn.Module):
    def __init__(self, in_channels=3, out_channels=25):
        super(SampleModel, self).__init__()

        # Create downsizing pass
        self.b1 = BasicBlock(3, 256, 256, expand=False)
        self.b2 = BasicBlock(256, 512, 512, expand=False)
        self.b3 = BasicBlock(512, 1024, 1024, expand=False)
        self.b4 = BasicBlock(1024, 2048, 2048, expand=False)
        
        # Create bottleneck
        self.bottleneck = nn.Conv2d(2048, 2048, 3, stride=1, padding='same')
        
        # Create upsizing pass
        self.b52 = BasicBlock(4096, 4096, 2048, expand=True)
        self.b5 = BasicBlock(2048, 2048, 1024, expand=True)
        self.extrapool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.b62 = BasicBlock(2048, 2048, 1024, expand=True)
        self.b6 = BasicBlock(1024, 1024, 512, expand=True)
        self.extrapool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.b72 = BasicBlock(1024, 1024, 512, expand=True)
        self.b7 = BasicBlock(512, 512, 256, expand=True)
        self.extrapool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.b82 = BasicBlock(512, 512, 256, expand=True)
        self.b8 = BasicBlock(256, 256, out_channels, expand=True)
        self.extrapool4 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        
    def forward(self, x):

        x1 = self.b1(x)
        x2 = self.b2(x1)
        x3 = self.b3(x2)
        x4 = self.b4(x3)
        
        x5 = self.bottleneck(x4)
        
        x6 = torch.cat((x4, x5), dim=1)
        x72 = self.b52(x6)
        x7 = self.b5(x72)
        x7e = self.extrapool1(x7)

        x8 = torch.cat((x3, x7e), dim=1)
        x92 = self.b62(x8)
        x9 = self.b6(x92)
        x9e = self.extrapool2(x9)

        x10 = torch.cat((x2, x9e), dim=1)
        x112 = self.b72(x10)
        x11 = self.b7(x112)
        x11e = self.extrapool3(x11)

        x12 = torch.cat((x1, x11e), dim=1)
        x132 = self.b82(x12)
        x13 = self.b8(x132)
        output = self.extrapool3(x13)

        return output

# 3. Training loop

In [12]:
def train_acc(model, epochs, optimizer, criterion):
    
    for epoch in range(epochs):
        
        train_losses = []
        train_accuracy = []
        val_losses = []
        val_accuracy = []
      
        ###### Train model ######
        model.train()
        for i, batch in enumerate(train_dataloader):
            # Extract images and masks
            img_batch, mask_batch = batch  # img [B,3,H,W], mask[B,H,W]
            img_batch, mask_batch = img_batch.to(device), mask_batch.long().to(device)
            
            # Optimize network
            optimizer.zero_grad()
            output = model(img_batch) # output: [B, 25, H, W]
            loss = criterion(output, mask_batch)
            wandb.log({"train_loss":loss.item()})
            loss.backward()
            optimizer.step()
          
            # Save batch results
            train_losses.append(loss.item())
            preds = torch.argmax(output, dim=1)
            acc = torch.sum(preds == mask_batch).item() / (mask_batch.shape[0] * mask_batch.shape[1] * mask_batch.shape[2])
            # we divide by (batch_size * height * width) to get average accuracy per pixel
            train_accuracy.append(acc)

        ###### Validate model ######
        model.eval()      
        for i, batch in enumerate(val_dataloader):
            #Extract data, labels
            img_batch, mask_batch = batch  # img [B,3,H,W], mask[B,H,W]
            img_batch, mask_batch = img_batch.to(device), mask_batch.long().to(device)

            #Validate model
            output = model(img_batch)
            loss = criterion(output, mask_batch)

            # Save batch results
            val_losses.append(loss.item())
            preds = torch.argmax(output, dim=1)
            acc = torch.sum(preds == mask_batch).item() / (mask_batch.shape[0] * mask_batch.shape[1] * mask_batch.shape[2])
            val_accuracy.append(acc)
            
        wandb.log({"train_acc":np.mean(train_accuracy), "val_acc":np.mean(val_accuracy), "val_loss":np.mean(val_losses)})
        ##### Print epoch results ######
        print(f'TRAIN       Epoch: {epoch} | Epoch metrics | loss: {np.mean(train_losses):.4f}, accuracy: {np.mean(train_accuracy):.3f}')        
        print(f'VALIDATION  Epoch: {epoch} | Epoch metrics | loss: {np.mean(val_losses):.4f}, accuracy: {np.mean(val_accuracy):.3f}')
        print('-' * 70)

With the dice metric

In [13]:
# With the dice metric
from torchmetrics.functional import dice as Dice

def train(model, epochs, optimizer, criterion):
    
    for epoch in range(epochs):
        
        train_losses = []
        train_dice = []
        train_accuracy = [] 
        val_losses = []
        val_dice = []
        val_accuracy = []
      
        ###### Train model ######
        model.train()
        for i, batch in enumerate(train_dataloader):
            # Extract images and masks
            img_batch, mask_batch = batch  # img [B,3,H,W], mask[B,H,W]
            img_batch, mask_batch = img_batch.to(device), mask_batch.long().to(device)
            
            # Optimize network
            optimizer.zero_grad()
            output = model(img_batch) # output: [B, 25, H, W]
            loss = criterion(output, mask_batch)
            wandb.log({"train_loss":loss.item()})
            loss.backward()
            optimizer.step()
          
            # Save batch results
            train_losses.append(loss.item())
            preds = torch.argmax(output, dim=1)
            dice = Dice(preds, mask_batch)
            train_dice.append(dice.cpu())
            acc = torch.sum(preds == mask_batch).item() / (mask_batch.shape[0] * mask_batch.shape[1] * mask_batch.shape[2])
            # we divide by (batch_size * height * width) to get average accuracy per pixel
            train_accuracy.append(acc)

        ###### Validate model ######
        model.eval()      
        for i, batch in enumerate(val_dataloader):
            #Extract data, labels
            img_batch, mask_batch = batch  # img [B,3,H,W], mask[B,H,W]
            img_batch, mask_batch = img_batch.to(device), mask_batch.long().to(device)

            #Validate model
            output = model(img_batch)
            loss = criterion(output, mask_batch)

            # Save batch results
            val_losses.append(loss.item())
            preds = torch.argmax(output, dim=1)
            acc = torch.sum(preds == mask_batch).item() / (mask_batch.shape[0] * mask_batch.shape[1] * mask_batch.shape[2])
            val_accuracy.append(acc)
            dice = Dice(preds, mask_batch)
            val_dice.append(dice.cpu())
        
        wandb.log({"train_acc":np.mean(train_accuracy), "val_acc":np.mean(val_accuracy), "val_loss":np.mean(val_losses), "val_dice": np.mean(val_dice),"train_dice": np.mean(train_dice)})
        ##### Print epoch results ######
        print(f'TRAIN       Epoch: {epoch} | Epoch metrics | loss: {np.mean(train_losses):.4f}, dice: {np.mean(train_dice):.3f}, accuracy: {np.mean(train_accuracy):.3f}')        
        print(f'VALIDATION  Epoch: {epoch} | Epoch metrics | loss: {np.mean(val_losses):.4f}, dice: {np.mean(val_dice):.3f}, accuracy: {np.mean(val_accuracy):.3f}')
        print('-' * 70)

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
epochs = 15
lr = 0.001
model = SampleModel(in_channels=3, out_channels=25).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss().to(device)

### Wandb

In [15]:
import wandb

wandb_api_key_label = "77e0a94a0fa993f825495a3021ebfa5477225ce9"

wandb.init(project="kaggle_competition", entity="leo-adri")

wandb.config = {
  "learning_rate": lr,
  "epochs": epochs,
  "batch_size": BATCH_SIZE
}

# Optional
wandb.watch(model)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


[]

In [16]:
train(model, epochs, optimizer, criterion)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


RuntimeError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 14.76 GiB total capacity; 13.43 GiB already allocated; 43.75 MiB free; 13.78 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
path = "/kaggle/working/Unet_aug-15"
torch.save(model.state_dict(), path)

# 4. Error Analysis

In [14]:
# Transfer model from gpu to cpu
model = model.cpu()

In [15]:
def visualise_model_performance(model: nn.Module, idx: int):
    
    # Load image
    img_paths = sorted(glob.glob(config.TRAIN_IMG_PATH + "/*"))
    mask_paths = sorted(glob.glob(config.TRAIN_MASK_PATH + "/*"))
    img_path = img_paths[idx]
    mask_path = mask_paths[idx]
    img = Image.open(img_path)
    true_mask = Image.open(mask_path)
    
    # Resize to the same shape as used in training
    img = train_dataset.transform_img(img)
    true_mask = train_dataset.transform_mask(true_mask)
    
    # Model accepts batches of images, not single images so we need to reshape tensors
    # From [C, H, W] to [B, C, H, W]
    img = img.unsqueeze(0)
    
    # Get predictions
    output = model(img)
    preds = torch.argmax(output, dim=1)
    
    # Tranform to numpy and reshape for plotting
    img = img.squeeze().permute(1,2,0).detach().numpy()
    preds = preds.permute(1,2,0).detach().numpy()
    true_mask = true_mask.permute(1,2,0).detach().numpy()
    
    # Plot
    f, axarr = plt.subplots(1,3, figsize=(15,5))
    axarr[0].imshow(img)
    axarr[1].imshow(preds, cmap='tab20')
    axarr[2].imshow(true_mask, cmap='tab20')
    axarr[0].set_title('Image')
    axarr[1].set_title('Model prediction')
    axarr[2].set_title('True mask')
    for ax in axarr:
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

In [17]:
visualise_model_performance(model, 10)
visualise_model_performance(model, 100)
visualise_model_performance(model, 200)

AttributeError: 'tensorDataset' object has no attribute 'transform_img'

The model is starting the get the hang of things but clearly there is still a lot of room for improvement.

# 5. Submission

In [ ]:
!mkdir /kaggle/working/test_predictions

In [ ]:
# Load and transform test images
test_img_link_list = sorted(glob.glob(config.TEST_IMG_PATH + '/*'))
torch_list = [val_dataset.transform_img(Image.open(link)) for link in test_img_link_list]

In [ ]:
model.eval()

# calculate output for each image in test set and save the prediction in new folder
for i in tqdm(range(len(torch_list))):
    img_id = test_img_link_list[i].split('/')[-1].split('.')[0]
    img = torch_list[i].unsqueeze(0)
    output = model(img)
    output = torch.argmax(output, dim=1).squeeze(0)
    output = np.uint8(output)
    output = Image.fromarray(output)
    output.save(f"/kaggle/working/test_predictions/{img_id}.png")

In [ ]:
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)


def create_rles():
    """Used for Kaggle submission: predicts and encode all test images"""
    dir = '/kaggle/working/test_predictions/'  # CHANGE DIRECTORY IF NEEDED
    N = len(list(os.listdir(dir)))
    with open('submission_file.csv', 'w') as f:
        f.write('ImageClassId,rle_mask\n')
        for index, i in enumerate(os.listdir(dir)):
            # print('{}/{}'.format(index, N))
                
            print(i)
            if i[0] != '.':
                mask = Image.open(dir + i)
                mask = mask.resize((1024, 1024), resample=Image.NEAREST)
                mask = np.array(mask)

                for x in range(1, 25):
                    enc = rle_encode(mask == x)
                    f.write(f"{i.split('_')[0]}_{x},{enc}\n")

create_rles()

# How to improve

The above solution is a very simple pipeline and gives a leaderboard score similar to the basline. From here, you can improve in various ways. This is a good practice in general: start with a simple baseline solution and try to improve from there. Below we outline a couple ideas you can try out but keep in mind that this is not an exhaustive list! It's often the unique ideas that give you an edge in Kaggle challenges.

### 1. Improving the speed of iteration
One of the best practices both in industry and on Kaggle is to create a pipeline in which you can iterate ideas quickly. It should take you no more than 20min (for this dataset) to train and evaluate a model. It doesn't mean that your final soultion should be trained in 20mins but it means that if you want to change some model parameters/data variants/training properties, you should be able to assess if your new idea is worth more of your efforts in about that much time. How you could speed up the above timeline:
- Include preprocessing step. Instead of loading large (4000x3000) images in the dataloader, resize them beforehand
- Increase number of workers in the dataloader
- Set up a better evaluation metric than accuracy. Ideally, it should be in line with the evaluation metric of the competition. This will allow your validation set to give a good approximation of your performance on the leaderboard
- Use GPUs for training. This can drive down your training time from hours to just minutes. Easy two options are: Google Colab (10 euro a month), or Kaggle (free but limit of 33 GPU hours per week - should be enough honestly)

### 2. Improving performance
Once you are able to iterate quickly here are a few ideas on how you could improve performance:
- Perform Exploratory Data Analysis (EDA). Think of the key problems with the dataset and how you could tackle them.
- Include transformations for you training set. You can use [native PyTorch transformations](https://pytorch.org/vision/stable/transforms.html) or I can recommend the [Albumentations library](https://albumentations.ai/) specifically designed for image transformations. Nice tutorial [here](https://www.youtube.com/watch?v=rAdLwKJBvPM)
- Try different architectures - Unet is a good start
- Improve your understanding of the training process. You could plot loss/accuracy/other metrics over time to get an idea of how many epochs you need to reach convergence. You could even use an experiment tracking software such as [Weights & Biases](https://wandb.ai/site)
- Experiment with different learning rates. You could even use a [learning rate scheduler](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate)
- Experiment with different loss functions
- Experiment with different image sizes
- Use transfer learning. This means you would need to obtain a semantic segmentation model from somewhere (no links here :) ) and replace the last layer to output 25 channels (because we have 25 classes in our dataset). Keep in mind that models pretrained on seemingly unrelated datasets like Imagenet can still work here! This is because the lower-level layers learn simple features such as lines, shapes, and colours, which can be useful for our problem as well.
- Think of a better train/validation split

<br>

This is already a lot of things to implement. Given that you have limited time, try to implement a few and maybe some of your own ideas and your leaderboard score surely will improve. Good luck!
